# 전처리 반영 코드

# data 불러오기
- import version 설정(터미널에서 지정)
    * pip install scikit-learn==1.1
    * pip install imbalanced-learn==0.9



## data 확인
데이터 출처: Kaggle
https://www.kaggle.com/code/joshuaswords/awesome-hr-data-visualization-prediction

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.ticker as mtick

# To ensure text placement and advanced grid layout
from matplotlib.gridspec import GridSpec

In [ ]:
# aug_test에는 target값이 없어서 aug_train만 사용할 예정
## 그리고 aug_train에서 train, validation, test set을 6:2:2로 나눌 예정
aug_train = pd.read_csv('data/aug_train.csv')

In [ ]:
aug_train.head(10)

In [ ]:
aug_train.columns

In [ ]:
aug_train.shape

### 결측치 확인
결측치가 꽤나 많은데 다 살리는 방향으로 갈 거다.
- enrollee_id                  0
- city                         0
- city_development_index       0
- gender                    4508
- relevent_experience          0
- enrolled_university        386
- education_level            460
- major_discipline          2813
- experience                  65
- company_size              5938
- company_type              6140
- last_new_job               423
- training_hours               0
- target                       0

In [ ]:
aug_train.isna().sum()

In [ ]:
# Check for nulls : 결측치 비율 확인
aug_train.isna().sum()/len(aug_train)

In [ ]:
# 결측치가 아예 없었던 데이터 10203개
# aug_train[aug_train.isnull().any(axis=1)]
aug_train.isna().sum(axis=1).astype(bool).sum()

### 실수형, 문자형 변수

In [ ]:
## 실수형, 문자형 변수
var_num = aug_train.select_dtypes(include=['float', 'int']).columns
var_obj = aug_train.select_dtypes(include=['object']).columns

print('features with dtype float : ', len(var_num),'개')
print(var_num)
print('features with dtype object : ', len(var_obj),'개')
print(var_obj)

In [ ]:
group_num = aug_train[['enrollee_id', 'city_development_index', 'training_hours', 'target']]
group_obj = aug_train[['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size',
                       'company_type', 'last_new_job']]

#### 실수형 변수
- 실수형 변수들은 결측치가 없다!

In [ ]:
group_num.isna().sum()

#### 범주형 자료 : categorical data 정리

In [ ]:
group_obj.isna().sum()

##### city
- city별로 one unique한 city_development_index 값을 가지기 때문에 city는 제거한다.

In [ ]:
# Group by 'city' and calculate the number of unique 'city_development_index' values
unique_development_index_counts = aug_train.groupby('city')['city_development_index'].nunique().reset_index()

# Check if any city has more than one unique city_development_index value
cities_with_multiple_development_indices = unique_development_index_counts[unique_development_index_counts['city_development_index'] > 1]

# Display the results
print(f"Cities with more than one unique 'city_development_index':\n{cities_with_multiple_development_indices}")

# Display the unique counts for all cities
print(f"\nUnique 'city_development_index' counts for all cities:\n{unique_development_index_counts}")

In [ ]:
aug_train['city'].unique()

In [ ]:
aug_train.drop(columns=['city'],inplace=True)

##### last_new_job 변수
결측치 처리 방법
1. (aug_train['company_type'].isna()) & (aug_train['company_size'].isna()) 조건에 맞는 행의 'last_new_job'을 'never'로 채움
    - never의 의미는 한 번도 직장을 구해보지 않았다는 의미다.
2. 이후의 남은 결측치는 NaN       230 => Unknown으로 채울 것

In [ ]:
aug_train[(aug_train['company_type'].isna())&(aug_train['company_size'].isna())]['last_new_job'].value_counts(dropna=False)

In [ ]:
# 필터링 조건에 맞는 행의 'last_new_job'을 'never'로 채움
mask = (aug_train['company_type'].isna()) & (aug_train['company_size'].isna())
aug_train.loc[mask, 'last_new_job'] = aug_train.loc[mask, 'last_new_job'].fillna('never')

In [ ]:
aug_train[(aug_train['company_type'].isna())&(aug_train['company_size'].isna())]['last_new_job'].value_counts(dropna=False)

In [ ]:
aug_train['last_new_job'].value_counts(dropna=False)

- NaN 230개는 나중에 experience 결측치 처리할 때 다시 보기

In [ ]:
aug_train['last_new_job'].fillna('Unknown',inplace=True)

In [ ]:
### 보기 좋게만 만들기
aug_train['last_new_job'] = aug_train['last_new_job'].apply(lambda x: 'Never' if x == 'never' else x) #just reads nicer


In [ ]:
aug_train['last_new_job'].value_counts(dropna=False)

##### experience 변수
1. 변수 value 값 처리
    - x: '0' if x == '<1' else x
        - 대신, 'experience' 열을 기준으로 'experience_under_1' 열 생성
    - x: '0' if x == '>20' else x
        - 대신, 'experience' 열을 기준으로 'experience_over_20' 열 생성
2. 결측치 : NaN      65개 발생
    - 'experience' 열을 기준으로 x가 결측치면 1, 아니면 0으로 'experience_Unknown' 열 생성
    - 'experience' 열에서 결측치는 '0'으로 대체


In [ ]:
aug_train['experience'].value_counts(dropna=False)

In [ ]:
# 'experience' 열을 기준으로 'experience_under_1' 열 생성
aug_train['experience_under_1'] = aug_train['experience'].apply(lambda x: 1 if x == '<1' else 0)

# 'experience' 열을 기준으로 'experience_over_20' 열 생성
aug_train['experience_over_20'] = aug_train['experience'].apply(lambda x: 1 if x == '>20' else 0)

In [ ]:
aug_train[aug_train['experience']=='0']['experience_over_20'].value_counts()

In [ ]:
aug_train[aug_train['experience']=='0']['experience_under_1'].value_counts()

In [ ]:
aug_train['experience_over_20'].value_counts()

In [ ]:
aug_train['experience_under_1'].value_counts()

In [ ]:
# 'experience' 열에서 '<1'을 '0'으로 대체
aug_train['experience'] = aug_train['experience'].apply(lambda x: '0' if x == '<1' else x)

# 'experience' 열에서 '>20'을 '0'으로 대체
aug_train['experience'] = aug_train['experience'].apply(lambda x: '0' if x == '>20' else x)

In [ ]:
# 'experience' 열을 기준으로 x가 결측치면 1, 아니면 0으로 'experience_Unknown' 열 생성
aug_train['experience_Unknown'] = aug_train['experience'].apply(lambda x: 1 if pd.isna(x) else 0)

# 'experience' 열에서 결측치는 '0'으로 대체
aug_train['experience'] = aug_train['experience'].fillna('0')

In [ ]:
aug_train['experience'].value_counts(dropna=False)

In [ ]:
aug_train['experience_Unknown'].value_counts(dropna=False)

##### company 변수
###### company_size 변수 
- 결측치 :NaN          5938 개,
###### company_type 변수 
- 결측치 :NaN                    6140개,
1. (1711개) company 변수 둘 다 결측치가 있고, last_new_job ='never'인 경우에는 company_type, company_size 둘 다 'Unemployed'
    - last_new_job : 이전 직업과 현재 직업의 몇 년 차이
    - 앞으로 다닐 회사가 아니라, last_new_job가 'never'인 즉, 이전회사가 없었던 얘들은 unemployed 된 애들이라고 생각함

2. 나머지는 'Unknown'으로 만들기
- aug_train['company_size'].fillna('Unknown',inplace=True)
- aug_train['company_type'].fillna('Unknown',inplace=True)
    - 결측치 겹치는 개 : 5360개 -> 얘네는 last_new_job과는 공통점이 없다.
    - company 정보가 현재 회사를 다니고 있고, 회사를 다니는 중에 교육을 받는다.

In [ ]:
aug_train['company_size'].value_counts(dropna=False)

In [ ]:
aug_train['company_type'].value_counts(dropna=False)

In [ ]:
aug_train[(aug_train['last_new_job']=='Never')&(aug_train['company_size'].isna())]['company_type'].value_counts(dropna=False)

In [ ]:
aug_train[(aug_train['last_new_job']=='Never')&(aug_train['company_type'].isna())]['company_size'].value_counts(dropna=False)

* Last_new_job이 never이면서 company size가 결측인게 없음
* 마찬가지로 last_new_job이 never이면서 company_type이 결측인게 없음

In [ ]:
# 필터링 조건 정의
mask = (aug_train['last_new_job']=='Never')&(aug_train['company_type'].isna())

# 'company_size'의 NaN 값을 'Unemployed'으로 채움
aug_train.loc[mask, 'company_size'] = aug_train.loc[mask, 'company_size'].fillna('Unemployed')

# 필터링 조건 정의
mask = (aug_train['last_new_job']=='Never')&(aug_train['company_size']=='Unemployed')
# 'company_type'의 NaN 값을 'Unemployed'으로 채움
aug_train.loc[mask, 'company_type'] = aug_train.loc[mask, 'company_type'].fillna('Unemployed')


In [ ]:
print(aug_train['company_type'].value_counts(dropna=False))
print(aug_train['company_size'].value_counts(dropna=False))

In [ ]:
# company_size 0으로 처리하는 걸로 바꿈.
aug_train['company_size'].fillna('0',inplace=True)
aug_train['company_type'].fillna('Unknown',inplace=True)

In [ ]:
### 보기 좋게만 만들기
aug_train['company_size'] = aug_train['company_size'].apply(lambda x: '10-49' if x == '10/49' else x) #diff replacement method

In [ ]:
print(aug_train['company_type'].value_counts(dropna=False))
print(aug_train['company_size'].value_counts(dropna=False))

##### Gender variable
- 결측치와 Other이랑 합친다.

In [ ]:
aug_train['gender'].fillna('Not provided',inplace=True)

In [ ]:
aug_train['gender'].value_counts(dropna=False)

In [ ]:
# 필터링 조건 수정: 'Not provided' 또는 'Other'
aug_train_gender = aug_train[(aug_train['gender'] == 'Not provided') | (aug_train['gender'] == 'Other')]
aug_train_gender['gender'].value_counts()

In [ ]:
group_num = aug_train[['enrollee_id', 'city_development_index', 'training_hours', 'target']]
group_obj = aug_train[['gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size',
                       'company_type', 'last_new_job']]

##### genderd의 'other'와 'not provided'의 분포 비교

In [ ]:
print('수치형 데이터 시각화')
# 수치형 데이터 시각화
for column in group_num:
    plt.figure(figsize=(8, 6))
    sns.histplot(data=aug_train_gender, x=column, hue='gender', multiple='dodge', palette='Set2')
    plt.title(f'{column} Distribution by Gender')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.legend(title='Gender', labels=['Other', 'Not provided'])
    plt.show()

print('범주형 데이터 시각화')
# 범주형 데이터 시각화
for column in group_obj:
    plt.figure(figsize=(8, 6))
    sns.countplot(data=aug_train_gender, x=column, hue='gender', palette='Set2')
    plt.title(f'{column} Distribution by Gender')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.legend(title='Gender', labels=['Not provided','Other'])
    plt.xticks(rotation=45)  # 범주형 데이터의 레이블이 길 경우 각도를 조정
    plt.show()


In [ ]:
# 'gender'가 'Not provided'인 행의 'gender'를 'Other'로 변경
aug_train.loc[aug_train['gender'] == 'Not provided', 'gender'] = 'Other'

In [ ]:
aug_train['gender'].value_counts(dropna=False)

##### Major variable
- 결측치와 Other이랑 합치려 했으나... education_level 에 분포의 차이가 있음
- 결측치는 'Unknown'으로, 'No Major'와 'Other' 도 각각 따로 가지고 가기로 결정

In [ ]:
aug_train['major_discipline'].fillna('Unknown',inplace=True)

In [ ]:
# 필터링 조건 수정: 'Unknown' 또는 'Other'
aug_train_major_discipline = aug_train[(aug_train['major_discipline'] == 'Unknown') | (aug_train['major_discipline'] == 'Other')]
aug_train_major_discipline['major_discipline'].value_counts(dropna= False)


##### major dicipline의 'other'와 'unknown'의 분포 비교

In [ ]:
group_num = aug_train[['enrollee_id', 'city_development_index', 'training_hours', 'target']]
group_obj = aug_train[['gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size',
                       'company_type', 'last_new_job']]

In [ ]:
print('수치형 데이터 시각화')
# 수치형 데이터 시각화
for column in group_num:
    plt.figure(figsize=(8, 6))
    sns.histplot(data=aug_train_major_discipline, x=column, hue='major_discipline', multiple='dodge', palette='Set2')
    plt.title(f'{column} Distribution by major_discipline')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.legend(title='Major Discipline', labels=['Other','Unknown'])
    plt.show()

print('범주형 데이터 시각화')
# 범주형 데이터 시각화
for column in group_obj:
    plt.figure(figsize=(8, 6))
    sns.histplot(data=aug_train_major_discipline, x=column, hue='major_discipline',multiple='dodge', palette='Set2')
    plt.title(f'{column} Distribution by major_discipline')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.legend(title='Major Discipline', labels=['Other', 'Unknown'])
    plt.show()

In [ ]:
aug_train['major_discipline'].value_counts(dropna=False)

##### enrolled_university
- 결측치 : NaN                   386개(0.020148, 비율) => 'Unknown'으로

In [ ]:
aug_train['enrolled_university'].value_counts(dropna=False)

In [ ]:
aug_train[aug_train['education_level'].isnull()]['enrolled_university'].value_counts(dropna=False)

In [ ]:
### 보기 좋게만 만들기
aug_train['enrolled_university'][aug_train['enrolled_university'] == 'no_enrollment'] = 'No Enrollment' #just reads nicer

In [ ]:
aug_train['enrolled_university'].fillna("Unknown", inplace=True)

In [ ]:
aug_train['enrolled_university'].value_counts(dropna=False)

##### education_level
- 결측치 460개,           0.024011 ==> Unknown

In [ ]:
aug_train['education_level'].value_counts(dropna=False)

In [ ]:
aug_train[aug_train['enrolled_university']=='Unknown']['education_level'].value_counts(dropna=False)

In [ ]:
aug_train.loc[aug_train['enrolled_university']=='no_enrollment','education_level'].value_counts(dropna=False)

In [ ]:
aug_train[aug_train['enrolled_university']=='No Enrollment']['education_level'].value_counts(dropna=False)

In [ ]:
aug_train['education_level'].fillna("Unknown", inplace=True)

In [ ]:
aug_train['education_level'].value_counts(dropna=False)

##### 결측치 있는 행 확인
- 결측치를 다 Unknown 살리고,
- Unknown 나름의 의미가 있다고 본다.

- 결측치로 인해서 데이터 행을 삭제하지 않고, 다 살려서 진행했다.

In [ ]:
aug_train.isna().sum()/len(aug_train)

##### 정수형으로 만들기

In [ ]:
print(aug_train.dtypes)

In [ ]:
# Converting objects to integers
aug_train['experience'] = aug_train['experience'].astype(str).astype(int)
aug_train['target'] = aug_train['target'].astype(int)

In [ ]:
print(aug_train.dtypes)

##### 변수 value 임의의 순서로 지정하기

In [ ]:
# 각 열의 고유 값 출력
for column in aug_train.columns:
    unique_values = aug_train[column].unique()
    print(f"Column '{column}' unique values: {unique_values}\n")

In [ ]:
# Orders
ed_order = ['Primary School','High School','Graduate','Masters','Phd','Unknown']
enroll_order = ['No Enrollment','Part time course','Full time course','Unknown']
disc_order = ['STEM','Humanities','Business Degree','Arts','Other','No Major','Unknown']
exp_yrs_order = ['0','<1','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','>20']
exp_yrs_order_2 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
size_order = ['0','<10', '10-49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+','Unemployed']
job_order = ['Never', '1', '2', '3', '4', '>4','Unknown']
exp_order =['No relevant experience','Has relevant experience']
gender_order = ['Male','Female','Other']
company_order = ['Pvt Ltd','Funded Startup','Public Sector','Early Stage Startup','NGO','Other','Unknown','Unemployed']

이 코드는 aug_train 데이터프레임을 사용하여 교육 수준(education_level)과 마지막 직업 변경 시기(last_new_job)에 따른 직업 경험(experience)을 분석하고, 직업을 찾는 사람과 그렇지 않은 사람의 비율을 비교하기 위해 데이터를 정규화하고 요약하는 과정을 포함하고 있습니다.

##### 각 변수별 유니크 값 확인하기

In [ ]:
aug_train.head()

In [ ]:
aug_train.columns

In [ ]:
aug_train.shape

* 14개 -> 총 33개 칼럼!

##### ------지선 -> 윤지 확인-----(지선 전처리에서 추가 코드된 부분이랑 수정된 부분 있어요!)----

#### 전처리 다하고 EDA 시작하기 전에, train, val, test 셋 분리하는 코드

In [ ]:
from sklearn.model_selection import train_test_split

X = aug_train.dropna().drop(columns=['target']).values
y = aug_train.dropna()['target'].values

# train, val, test 셋 분리하는 코드
# 데이터셋 분할: Train/Validation + Test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=80)
# 데이터셋 분할: Train + Validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=80)

In [ ]:
# 열 이름 설정
feature_columns = aug_train.drop(columns=['target']).columns
target_column = ['target']

# numpy 배열을 DataFrame으로 변환
X_train_df = pd.DataFrame(X_train_val, columns=feature_columns)
X_val_df = pd.DataFrame(X_val, columns=feature_columns)
X_test_df = pd.DataFrame(X_test, columns=feature_columns)
y_train_df = pd.DataFrame(y_train_val, columns=target_column)
y_val_df = pd.DataFrame(y_val, columns=target_column)
y_test_df = pd.DataFrame(y_test, columns=target_column)

# Ensure 'train_df' and 'test_df' are created correctly by concatenating X and y
# train_df : for visuals
train_df = pd.concat([X_train_df, y_train_df], axis=1)
test_df = pd.concat([X_val_df, y_val_df], axis=1)
new_data_df = pd.concat([X_test_df, y_test_df], axis=1)

# 출력 확인 : for visual
print(train_df.info())
print(test_df.info())

In [ ]:
X_train, X_test, y_train, y_test = X_train, X_val, y_train, y_val

#### EDA 전, unique한 값 확인
- 일단 모든 변수 object형이다.

In [ ]:
#relevent_experience
aug_train['relevent_experience'].unique()

In [ ]:
#
aug_train['enrolled_university'].unique()

In [ ]:
#
aug_train['education_level'].unique()

In [ ]:
#
aug_train['major_discipline'].unique()

In [ ]:
#
aug_train['experience'].unique()

In [ ]:
aug_train['company_size'].unique()

In [ ]:
aug_train['company_type'].unique()

In [ ]:
aug_train['last_new_job'].unique()

In [ ]:
aug_train['last_new_job'].unique()

In [ ]:
aug_train['training_hours'].unique()

## 기존 EDA 정리

In [ ]:
sns.palplot(['#002d1d','#0e4f66','gray','#fbfbfb'])

In [ ]:
x=train_df.groupby(['target'])['target'].count()
y=len(train_df)
r=((x/y)).round(2)
ratio = pd.DataFrame(r).T


fig, ax = plt.subplots(1,1,figsize=(6.5, 2),dpi=150)
background_color = "#fbfbfb"
fig.patch.set_facecolor(background_color)
ax.set_facecolor(background_color) 

ax.barh(ratio.index, ratio[1.0], color='#0e4f66', alpha=0.9, ec=background_color, label='Job-Seeker')
ax.barh(ratio.index, ratio[0.0], left=ratio[1.0], color='gray', alpha=0.9,ec=background_color, label='Non Job-Seeker')

ax.set_xlim(0, 1)
ax.set_xticks([])
ax.set_yticks([])
ax.legend().set_visible(False)
for s in ['top', 'left', 'right', 'bottom']:
    ax.spines[s].set_visible(False)
    
for i in ratio.index:
    ax.annotate(f"{int(ratio[1.0][i]*100)}%", xy=(ratio[1.0][i]/2, i),va = 'center', ha='center',fontsize=32, fontweight='light', fontfamily='serif',color='white')
    ax.annotate("Job-Seeker", xy=(ratio[1.0][i]/2, -0.25),va = 'center', ha='center',fontsize=12, fontweight='light', fontfamily='serif',color='white')
    
    
for i in ratio.index:
    ax.annotate(f"{int(ratio[0.0][i]*100)}%", xy=(ratio[1.0][i]+ratio[0.0][i]/2, i),va = 'center', ha='center',fontsize=32, fontweight='light', fontfamily='serif',color='white')
    ax.annotate("Non Job-Seeker", xy=(ratio[1.0][i]+ratio[0.0][i]/2, -0.25),va = 'center', ha='center',fontsize=12, fontweight='light', fontfamily='serif',color='white')


fig.text(0.125,1.1,'How many are looking for a new role?', fontfamily='serif',fontsize=15, fontweight='bold')
fig.text(0.125,0.915,'We see an imbalanced dataset;\nmost people are not job-seeking',fontfamily='serif',fontsize=12)  


plt.show()

In [ ]:
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)

In [ ]:
background_color = "#fbfbfb"

fig = plt.figure(figsize=(22,15),dpi=150)
fig.patch.set_facecolor(background_color)
gs = fig.add_gridspec(3, 3)
gs.update(wspace=0.35, hspace=0.27)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[0, 2])
ax3 = fig.add_subplot(gs[1, 0])
ax4 = fig.add_subplot(gs[1, 1])
ax5 = fig.add_subplot(gs[1, 2])
ax6 = fig.add_subplot(gs[2, 0])
ax7 = fig.add_subplot(gs[2, 1])
ax8 = fig.add_subplot(gs[2, 2])


# Ax0 - EDUCATION LEVEL
train = pd.DataFrame(train_df["education_level"].value_counts())
train["Percentage"] = train["education_level"].apply(lambda x: x/sum(train["education_level"])*100)
train = train.sort_index()

test = pd.DataFrame(test_df["education_level"].value_counts())
test["Percentage"] = test["education_level"].apply(lambda x: x/sum(test["education_level"])*100)
test = test.sort_index()

ax0.bar(np.arange(len(train.index)), height=train["Percentage"], zorder=3, color="gray", width=0.05)
ax0.scatter(np.arange(len(train.index)), train["Percentage"], zorder=3,s=200, color="gray")
ax0.bar(np.arange(len(test.index))+0.4, height=test["Percentage"], zorder=3, color="#0e4f66", width=0.05)
ax0.scatter(np.arange(len(test.index))+0.4, test["Percentage"], zorder=3,s=200, color="#0e4f66")
ax0.text(-0.5, 68.5, 'Education Level', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
ax0.yaxis.set_major_formatter(mtick.PercentFormatter())
ax0.yaxis.set_major_locator(mtick.MultipleLocator(10))
ax0.set_xticks(np.arange(len(train.index))+0.4 / 2)
ax0.set_xticklabels(list(train.index),rotation=0)


# Ax1 - ENROLLED IN UNIVESITY
train = pd.DataFrame(train_df["enrolled_university"].value_counts())
train["Percentage"] = train["enrolled_university"].apply(lambda x: x/sum(train["enrolled_university"])*100).loc[enroll_order]
test = pd.DataFrame(test_df["enrolled_university"].value_counts())
test["Percentage"] = test["enrolled_university"].apply(lambda x: x/sum(test["enrolled_university"])*100).loc[enroll_order]

ax1.text(0, 2.5, 'University Enrollment', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
ax1.barh(train.index, train['Percentage'], color="gray", zorder=3, height=0.6)
ax1.barh(test.index, test['Percentage'], color="#0e4f66", zorder=3, height=0.4)
ax1.xaxis.set_major_formatter(mtick.PercentFormatter())
ax1.xaxis.set_major_locator(mtick.MultipleLocator(10))

###
# Ax2 - GENDER 
train = pd.DataFrame(train_df["gender"].value_counts())
train["Percentage"] = train["gender"].apply(lambda x: x/sum(train["gender"])*100)
test = pd.DataFrame(test_df["gender"].value_counts())
test["Percentage"] = test["gender"].apply(lambda x: x/sum(test["gender"])*100)

x = np.arange(len(train))
ax2.text(-0.6, 76, 'Gender', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
ax2.grid(color='gray', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
ax2.bar(x, height=train["Percentage"], zorder=3, color="gray", width=0.4)
ax2.bar(x+0.4, height=test["Percentage"], zorder=3, color="#0e4f66", width=0.4)
ax2.set_xticks(x + 0.4 / 2)
ax2.set_xticklabels(['Male','Female','Other'])
ax2.yaxis.set_major_formatter(mtick.PercentFormatter())
ax2.yaxis.set_major_locator(mtick.MultipleLocator(10))
for i,j in zip([0, 1], train["Percentage"]):
    ax2.annotate(f'{j:0.0f}%',xy=(i, j/2), color='#f6f6f6', horizontalalignment='center', verticalalignment='center')
for i,j in zip([0, 1], test["Percentage"]):
    ax2.annotate(f'{j:0.0f}%',xy=(i+0.4, j/2), color='#f6f6f6', horizontalalignment='center', verticalalignment='center')
    

    
## Ax 3 - CDI

ax3.grid(color='gray', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
train = pd.DataFrame(train_df["city_development_index"])
test = pd.DataFrame(test_df["city_development_index"])
sns.kdeplot(train["city_development_index"], ax=ax3,color="gray", shade=True, label="Train")
sns.kdeplot(test["city_development_index"], ax=ax3, color="#0e4f66", shade=True, label="Test")
ax3.text(0.29, 13, 'City Development Index', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
ax3.yaxis.set_major_locator(mtick.MultipleLocator(2))
ax3.set_ylabel('')    
ax3.set_xlabel('')

## AX4 - TITLE

ax4.spines["bottom"].set_visible(False)
ax4.tick_params(left=False, bottom=False)
ax4.set_xticklabels([])
ax4.set_yticklabels([])
ax4.text(0.5, 0.6, 'How do our\n\n datasets compare?', horizontalalignment='center', verticalalignment='center',fontsize=22, fontweight='bold', fontfamily='serif', color="#323232")
ax4.text(0.28,0.57,"Train", fontweight="bold", fontfamily='serif', fontsize=22, color='gray')
ax4.text(0.5,0.57,"&", fontweight="bold", fontfamily='serif', fontsize=22, color='#323232')
ax4.text(0.58,0.57,"Test", fontweight="bold", fontfamily='serif', fontsize=22, color='#0e4f66')




### Ax5 - RELEVANT EXPERIENCE
train = pd.DataFrame(train_df["relevent_experience"].value_counts())
train["Percentage"] = train["relevent_experience"].apply(lambda x: x/sum(train["relevent_experience"])*100)
test = pd.DataFrame(test_df["relevent_experience"].value_counts())
test["Percentage"] = test["relevent_experience"].apply(lambda x: x/sum(test["relevent_experience"])*100)
x = np.arange(len(train))
ax5.text(-0.4, 80, 'Relevant Experience', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
ax5.grid(color='gray', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
ax5.bar(x, height=train["Percentage"], zorder=3, color="gray", width=0.4)
ax5.bar(x+0.4, height=test["Percentage"], zorder=3, color="#0e4f66", width=0.4)
ax5.set_xticks(x + 0.4 / 2)
ax5.set_xticklabels(['No relevant experience','Has relevant experience'])
ax5.yaxis.set_major_formatter(mtick.PercentFormatter())
ax5.yaxis.set_major_locator(mtick.MultipleLocator(10))

for i,j in zip([0, 1, 2], train["Percentage"]):
    ax5.annotate(f'{j:0.0f}%',xy=(i, j/2), color='#f6f6f6', horizontalalignment='center', verticalalignment='center')
for i,j in zip([0, 1, 2], test["Percentage"]):
    ax5.annotate(f'{j:0.0f}%',xy=(i+0.4, j/2), color='#f6f6f6', horizontalalignment='center', verticalalignment='center')

    
    
# Ax6 - TRAINING HOURS
train = pd.DataFrame(train_df["training_hours"])
train["TrainTest"] = "Train"
test = pd.DataFrame(test_df["training_hours"])
test["TrainTest"] = "Test"
ax6.text(-0.65, 370, 'Training Hours', fontsize=14, fontweight='bold', fontfamily='serif', color="#002d1d")
comb_graph_temp_df = pd.concat([train, test], axis=0)
sns.boxenplot(ax=ax6, y="training_hours", x="TrainTest", data=comb_graph_temp_df, palette=["gray", "#0e4f66"])
ax6.set_xlabel("")
ax6.set_ylabel("")


# Ax7 - EXPERIENCE YRS
train = pd.DataFrame(train_df["experience"].value_counts())
train["Percentage"] = train["experience"].apply(lambda x: x/sum(train["experience"])*100)
train = train.sort_index()
test = pd.DataFrame(test_df["experience"].value_counts())
test["Percentage"] = round(test["experience"].apply(lambda x: x/sum(test["experience"])*100),).astype(int)
test = test.sort_index()
ax7.grid(color='gray', linestyle=':', axis='y', zorder=0,  dashes=(1,5))
ax7.plot(train.index, train["Percentage"], zorder=3, color="gray", marker='o')
ax7.plot(test.index, test["Percentage"], zorder=3, color="#0e4f66", marker='o')
ax7.text(-1.5, 20.5, 'Years Experience', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
ax7.set_yticklabels(labels = ['0   ', '5%','10%','15%'])
ax7.xaxis.set_major_locator(mtick.MultipleLocator(5))
ax7.yaxis.set_major_locator(mtick.MultipleLocator(5))


# Ax8 - MAJOR DISCIPLINE
train = pd.DataFrame(train_df["major_discipline"].value_counts())
train["Percentage"] = train["major_discipline"].apply(lambda x: x/sum(train["major_discipline"])*100)
test = pd.DataFrame(test_df["major_discipline"].value_counts())
test["Percentage"] = test["major_discipline"].apply(lambda x: x/sum(test["major_discipline"])*100)

ax8.barh(np.arange(len(train.index)), train["Percentage"], zorder=3, color="gray", height=0.4)
ax8.barh(np.arange(len(test.index))+0.4, test["Percentage"], zorder=3, color="#0e4f66", height=0.4)
ax8.text(-5, -0.8, 'Major Discipline', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
ax8.xaxis.set_major_formatter(mtick.PercentFormatter())
ax8.yaxis.set_major_locator(mtick.MultipleLocator(1))
ax8.set_yticks(np.arange(len(test.index))+0.4 / 2)
ax8.set_yticklabels(list(test.index))
ax8.invert_yaxis()



for i in range(0,9):
    locals()["ax"+str(i)].set_facecolor(background_color) 
    
for i in range(0,9):
    locals()["ax"+str(i)].tick_params(axis=u'both', which=u'both',length=0)


for s in ["top","right","left"]:
    for i in range(0,9):
        locals()["ax"+str(i)].spines[s].set_visible(False)

plt.show()

In [ ]:
color_palette=["gray","#0e4f66"]
fig = plt.figure(figsize=(18,15), dpi=150)
fig.patch.set_facecolor(background_color) # figure background color
gs = fig.add_gridspec(3, 3)
gs.update(wspace=0.4, hspace=0.6)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax4 = fig.add_subplot(gs[2, 0])
ax5 = fig.add_subplot(gs[2, 1])

# Distribution
ax0.text(-1, 19000, 'Who is looking for a new job?', fontsize=20, fontweight='bold', fontfamily='serif', color="#323232")
ax0.text(-1, 17500, 'Most job-seekers appear to be male', fontsize=14, fontweight='light', fontfamily='serif', color="#323232")
ax0.text(-1, 14050, 'Overall', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
sns.countplot(x=train_df["gender"], color="#247747", ax=ax0, zorder=3,alpha=0.9)



# Gender
ax1.text(-1, 11000, 'Job searching by gender', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
sns.countplot(x="gender", hue="target", data=train_df, palette=color_palette, ax=ax1, zorder=3)
legend_labels, _= ax1.get_legend_handles_labels()
ax1.legend(legend_labels, ["Non-Job Seeker", "Job Seeker"], ncol=2, bbox_to_anchor=(-0.52, 1.28), facecolor=background_color, edgecolor=background_color)



# CDI
ax2.text(0.3, 16, 'Does the City Development Index play a role?', fontsize=20, fontweight='bold', fontfamily='serif', color="#323232")
ax2.text(0.3, 14.5, 'Interestingly, we see Job Seekers are frequently from cities with a lower CDI score', fontsize=14, fontweight='light', fontfamily='serif', color="#323232")
ax2.text(0.3, 13, 'Overall', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
sns.kdeplot(train_df["city_development_index"], color="#247747", shade=True, ax=ax2, zorder=3)



ax3.text(0.33, 15.5, 'Job Seeker / Non-Job Seeker', fontsize=14, fontweight='bold', fontfamily='serif', color="#323232")
sns.kdeplot(train_df.loc[(train_df["target"]==0), "city_development_index"], color="gray", label="Not Survived", ax=ax3)
sns.kdeplot(train_df.loc[(train_df["target"]==1), "city_development_index"], color="#0e4f66", label="Survived", ax=ax3)



###
aug_train['count'] = 1
job_hunt_only = aug_train[aug_train['target']==1]
no_job_hunt_only = aug_train[aug_train['target']==0]

job_change = aug_train.groupby(['education_level','last_new_job'])['experience'].sum().unstack().loc[ed_order,job_order]

job_hunt_only.groupby(['target','last_new_job'])['count'].sum().unstack()
notseek_job_change = no_job_hunt_only.groupby(['target','last_new_job'])['count'].sum().unstack().T
seek_job_change = job_hunt_only.groupby(['target','last_new_job'])['count'].sum().unstack().T

notseek_job_change.columns = ['count']
seek_job_change.columns = ['count']

notseek_job_change["percentage"] = notseek_job_change["count"].apply(lambda x: x/sum(notseek_job_change["count"])) *100
seek_job_change["percentage"] = seek_job_change["count"].apply(lambda x: x/sum(seek_job_change["count"])) *100


ed_notseek_job_change = no_job_hunt_only.groupby(['target','education_level'])['count'].sum().unstack().T.loc[ed_order]
ed_seek_job_change = job_hunt_only.groupby(['target','education_level'])['count'].sum().unstack().T.loc[ed_order]

ed_notseek_job_change.columns = ['count']
ed_seek_job_change.columns = ['count']

ed_notseek_job_change["percentage"] = ed_notseek_job_change["count"].apply(lambda x: x/sum(ed_notseek_job_change["count"])) *100
ed_seek_job_change["percentage"] = ed_seek_job_change["count"].apply(lambda x: x/sum(ed_seek_job_change["count"])) *100

###


ax4.barh(notseek_job_change.index, notseek_job_change['percentage'], color="gray", zorder=3, height=0.7)
ax4.barh(seek_job_change.index, seek_job_change['percentage'], color="#0e4f66", zorder=3, height=0.3)
ax4.xaxis.set_major_locator(mtick.MultipleLocator(10))


##
ax5.barh(ed_notseek_job_change.index, ed_notseek_job_change['percentage'], color="gray", zorder=3, height=0.7)
ax5.barh(ed_seek_job_change.index, ed_seek_job_change['percentage'], color="#0e4f66", zorder=3, height=0.3)
ax5.xaxis.set_major_locator(mtick.MultipleLocator(10))

##
ax4.text(-1, 5.7, 'Last job change (yrs)',fontsize=15, fontweight='bold', fontfamily='serif',color='#323232')
ax5.text(0, 4.55, 'Education level', fontsize=15, fontweight='bold', fontfamily='serif',color='#323232')

ax4.text(-2.5, 7.5, 'Are there other differences?', 
         fontsize=20, fontweight='bold', fontfamily='serif',color='#323232')

ax4.text(-2.5, 6.75, 
         'We see broadly similar patterns, but notable areas of difference', 
         fontsize=14, fontweight='light', fontfamily='serif')


####

fig.text(0.77, 0.89
         , 'Insight', fontsize=15, fontweight='bold', fontfamily='serif',color='#323232')

fig.text(0.77, 0.39, '''
We note that most job-seekers are Male.
This is not all that surprising as in this
dataset Males make up the majority of the
sample population.

What is more interesting though is
the City Development Index (CDI) chart.
There we see that there are two peaks
for job-seekers. 
The peaks are at high and low CDI scores. 


We can ponder why this might be;
in high CDI areas perhaps there are a 
lot of opportunities and therefore
people feel encouraged to seek better roles.


Perhaps in lower CDI areas candidates
want to improve their circumstances by
searching for new jobs, maybe in new areas. 


This is all conjecture, but interesting
nonetheless.

It is also interesting to see that job-seekers
have changed job more often that non-job seekers
within that past 1 year, and also
those that have never looked for a job
also seem to be ready for a new
challenge.
'''
         , fontsize=14, fontweight='light', fontfamily='serif',color='#323232')



import matplotlib.lines as lines
l1 = lines.Line2D([0.7, 0.7], [0.1, 0.9], transform=fig.transFigure, figure=fig,color='black',lw=0.2)
fig.lines.extend([l1])

for s in ["top","right","left"]:
    for i in range(0,6):
        locals()["ax"+str(i)].spines[s].set_visible(False)
        
for i in range(0,6):
        locals()["ax"+str(i)].set_facecolor(background_color)
        locals()["ax"+str(i)].tick_params(axis=u'both', which=u'both',length=0)
        locals()["ax"+str(i)].grid(color='gray', linestyle=':', axis='y', zorder=0,  dashes=(1,5))      

        
for x in range(0,4):
    for y in range(0,4):
        locals()["ax"+str(x)].set_xlabel("")
        locals()["ax"+str(y)].set_ylabel("")

plt.show()

In [ ]:
pv_gen_size = pd.pivot_table(aug_train, values='count',index=['gender'],columns=['company_size'],aggfunc=np.sum).loc[gender_order, size_order]
ct_gen_size = pd.crosstab(aug_train['company_size'],aug_train['experience'], normalize='index').loc[size_order,exp_yrs_order_2]


fig = plt.figure(figsize=(15,10), dpi=150) 
fig.patch.set_facecolor(background_color)
gs = fig.add_gridspec(2, 1)
gs.update(wspace=0, hspace=-0.09)
ax0 = fig.add_subplot(gs[:,:])

colors = ["#fbfbfb", "#4b4b4c","#0e4f66"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

sns.heatmap(ax=ax0, data=ct_gen_size, linewidths=.1, vmin=0, vmax=0.075,
            square=True, cbar=False, cmap=colormap,linewidth=3, annot=True, fmt='1.0%',annot_kws={"fontsize":14})

 
ax0.set_facecolor(background_color) 
ax0.set_xlabel("Employee Experience [Years]",fontfamily='serif',fontsize=12,loc='left')
ax0.set_ylabel("")

for s in ["top","right","left"]:
    ax0.spines[s].set_visible(False)

    
ax0.text(0, -1.4, 
         'Company size & employee experience', 
         fontsize=20, 
         fontweight='bold', 
         fontfamily='serif',
        )

ax0.text(0, -0.9, 
         'Those with over 20 yrs experience dominate the work force at all company sizes.', 
         fontsize=13, 
         fontweight='light', 
         fontfamily='serif',
        )
ax0.text(0, -0.5, 
         'We also observe some heat around the lower experience range and at smaller companies.', 
         fontsize=13, 
         fontweight='light', 
         fontfamily='serif',
        )

ax0.tick_params(axis=u'both', which=u'both',length=0)
plt.show()

In [ ]:
job_hunt_only = aug_train[aug_train['target']==1]
no_job_hunt_only = aug_train[aug_train['target']==0]

job_seek = pd.crosstab(job_hunt_only['company_size'],job_hunt_only['experience'], normalize='index').loc[size_order,exp_yrs_order_2]
no_seek = pd.crosstab(no_job_hunt_only['company_size'],no_job_hunt_only['experience'], normalize='index').loc[size_order,exp_yrs_order_2]

###
fig = plt.figure(figsize=(14,14),dpi=150)
fig.patch.set_facecolor(background_color)
gs = fig.add_gridspec(2, 3)
gs.update(wspace=0.2, hspace=0.3)
ax0 = fig.add_subplot(gs[0,:])
ax1 = fig.add_subplot(gs[1,:])


colors = ["#fbfbfb", "#4b4b4c","#0e4f66"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

sns.heatmap(ax=ax0, data=job_seek, linewidths=.1, vmin=0, vmax=0.075,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False, cmap=colormap,linewidth=3, annot=False, fmt='1.0%',annot_kws={"fontsize":14})

sns.heatmap(ax=ax1, data=no_seek, linewidths=.1, vmin=0, vmax=0.075,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False, cmap=colormap,linewidth=3, annot=False, fmt='1.0%',annot_kws={"fontsize":14})

ax0.set_facecolor(background_color) 
ax0.set_xlabel("")
ax0.set_ylabel("")

for s in ["top","right","left"]:
    ax0.spines[s].set_visible(False)

    
ax0.set_xlabel("")
ax0.set_ylabel("")
ax1.set_xlabel("Employee Experience [Years]",fontfamily='serif',fontsize=14,loc='left')
ax1.set_ylabel("")

for s in ["top","right","left"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)

    
ax0.text(0, -1.3, 
         'Job Seekers: Company size & employee experience', 
         fontsize=20, 
         fontweight='bold', 
         fontfamily='serif',
        )

ax1.text(0, -0.7, 
         'Those with over 20 yrs experience dominate the work force at all company sizes.', 
         fontsize=13, 
         fontweight='light', 
         fontfamily='serif',
        )
ax1.text(0, -0.35, 
         'People with 20+ years of experience are not seeking new roles; perhaps they are at their desired seniority level.', 
         fontsize=13, 
         fontweight='light', 
         fontfamily='serif',
        )
    
ax1.text(0, -1.3, 
         'Non-Job Seekers: Company size & employee experience', 
         fontsize=20, 
         fontweight='bold', 
         fontfamily='serif',
        )

ax0.text(0, -0.7, 
         'We see that most job seekers have between 2 - 6 years experience.', 
         fontsize=13, 
         fontweight='light', 
         fontfamily='serif',
        )
ax0.text(0, -0.35, 
         'Anecdotally this seems correct; people build their skills in the first few years of their career then seek new challenges.', 
         fontsize=13, 
         fontweight='light', 
         fontfamily='serif',
        )

ax0.tick_params(axis=u'both', which=u'both',length=0)
ax1.tick_params(axis=u'both', which=u'both',length=0)

plt.show()

In [ ]:
job_hunt_only = aug_train[aug_train['target']==1]
no_job_hunt_only = aug_train[aug_train['target']==0]

job_seek = pd.crosstab(job_hunt_only['education_level'],job_hunt_only['company_type'], normalize='index').loc[ed_order,company_order]
no_seek = pd.crosstab(no_job_hunt_only['education_level'],no_job_hunt_only['company_type'], normalize='index').loc[ed_order,company_order]


fig = plt.figure(figsize=(15,15),dpi=150)
fig.patch.set_facecolor(background_color)

gs = fig.add_gridspec(1, 2)
gs.update(wspace=0.05, hspace=0.3)
ax0 = fig.add_subplot(gs[0,0])
ax1 = fig.add_subplot(gs[0,1])


colors = ["#fbfbfb", "#4b4b4c","#0e4f66"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)

sns.heatmap(ax=ax0, data=job_seek, linewidths=.1, vmin=-0.01, vmax=0.075,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False, cmap=colormap,linewidth=3, annot=True, fmt='1.0%',annot_kws={"fontsize":14})

sns.heatmap(ax=ax1, data=no_seek, linewidths=.1, vmin=-0.01, vmax=0.075,
            square=True, cbar_kws={"orientation": "horizontal"}, cbar=False,yticklabels=False, cmap=colormap,linewidth=3, annot=True, fmt='1.0%',annot_kws={"fontsize":14})


ax0.set_facecolor(background_color)
ax1.set_facecolor(background_color) 
ax0.set_xlabel("")
ax0.set_ylabel("")

for s in ["top","right","left"]:
    ax0.spines[s].set_visible(False)


ax1.set_xlabel("")
ax1.set_ylabel("")

for s in ["top","right","left"]:
    ax0.spines[s].set_visible(False)
    ax1.spines[s].set_visible(False)
    
    
ax0.text(0, -1, 
         'Education level & company type', 
         fontsize=20, 
         fontweight='bold', 
         fontfamily='serif',
        )

ax0.text(0, -0.6, 
         'Job Seekers', 
         fontsize=15, 
         fontweight='bold', 
         fontfamily='serif',
        )

ax0.text(0, -0.2, 
         'Seem to be at earlier stages of their education.', 
         fontsize=13, 
         fontweight='light', 
         fontfamily='serif',
        )


###

ax1.text(0, -0.6, 
         'Non-Job Seekers', 
         fontsize=15, 
         fontweight='bold', 
         fontfamily='serif',
        )
ax1.text(0, -0.2, 
         'We see a slightly higher education level.', 
         fontsize=13, 
         fontweight='light', 
         fontfamily='serif',
        )
    

ax0.tick_params(axis=u'both', which=u'both',length=0)
ax1.tick_params(axis=u'both', which=u'both',length=0)
plt.show()

In [ ]:
# Orders
#ed_order = ['Primary School','High School','Graduate','Masters','Phd']
#enroll_order = ['No Enrollment','Part time course','Full time course']
#disc_order = ['STEM','Unknown','Humanities','Other','Business Degree','Arts','No Major']
#exp_yrs_order = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20']
#size_order = ['0','<10', '10-49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+']
#job_order = ['Never', '1', '2', '3', '4', '>4']
#exp_order =['No relevant experience','Has relevant experience']
#gender_order = ['Male','Female','Other','Not provided']
#company_order = ['Pvt Ltd','Unknown','Funded Startup','Public Sector','Early Stage Startup','NGO','Other']

In [ ]:
data = train_df
data['count'] = 1

data_plot = pd.pivot_table(data, values='count', index=['major_discipline'], columns=['company_size'], aggfunc=np.sum).fillna(0).astype(int).stack().loc[disc_order, size_order]
data_job_seek = pd.pivot_table(data[data['target']==1], values='count', index=['major_discipline'], columns=['company_size'], aggfunc=np.sum).fillna(0).astype(int).stack().loc[disc_order, size_order]
data_no_job_seek = pd.pivot_table(data[data['target']==0], values='count', index=['major_discipline'], columns=['company_size'], aggfunc=np.sum).fillna(0).astype(int).stack().loc[disc_order, size_order]

In [ ]:
# https://stackoverflow.com/questions/56337732/how-to-plot-scatter-pie-chart-using-matplotlib
def drawPieMarker(xs, ys, ratios, sizes, colors, ax):
    markers = []
    previous = 0
    # calculate the points of the pie pieces
    for color, ratio in zip(colors, ratios):
        this = 2 * np.pi * ratio + previous
        x  = [0] + np.cos(np.linspace(previous, this, 30)).tolist() + [0]
        y  = [0] + np.sin(np.linspace(previous, this, 30)).tolist() + [0]
        xy = np.column_stack([x, y])
        previous = this
        markers.append({'marker':xy, 's':np.abs(xy).max()**2*np.array(sizes), 'facecolor':color})

    # scatter each of the pie pieces to create pies
    for marker in markers:
        ax.scatter(xs, ys, **marker, alpha=0.9, ec=background_color)

In [ ]:
# Plot inspired by Subin An

fig = plt.figure(figsize=(13, 13), dpi=150)
gs = fig.add_gridspec(5, 5)
gs.update(wspace=0.1, hspace=0.1)
fig.patch.set_facecolor(background_color) 

# Pie 

ax_centre = fig.add_subplot(gs[1:4, 0:4]) 
for cl_idx in disc_order[::-1]:
    for age_idx in size_order:
        seek = data_job_seek[cl_idx][age_idx]
        no_seek = data_no_job_seek[cl_idx][age_idx]
        total = data_job_seek[cl_idx][age_idx]
        drawPieMarker([age_idx],[cl_idx], [seek/(seek+no_seek), no_seek/(seek+no_seek)] ,[total*2.5], ["#0e4f66", "gray"], ax=ax_centre)

ax_centre.grid(linewidth=0.1)        
ax_centre.set_facecolor(background_color)
ax_centre.set_xticklabels(size_order,fontfamily='serif', fontsize=11, rotation=90)

 # Top
ax_top = fig.add_subplot(gs[0, :4], sharex=ax_centre) 
c_size_non = data[data['target']==0]['company_size'].value_counts()[size_order]
ax_top.bar(c_size_non.index, c_size_non, width=0.45, alpha=0.9,ec=background_color, color='gray')

c_size = data[data['target']==1]['company_size'].value_counts()[size_order]
ax_top.bar(c_size.index, c_size, bottom=c_size_non , width=0.45, alpha=0.9, ec=background_color,color='#0e4f66')

plt.setp(ax_top.get_xticklabels(), visible=False)
ax_top.set_facecolor(background_color)

# Side 
ax_side = fig.add_subplot(gs[1:4, 4], sharey=ax_centre) 
disc_no = data[data['target']==0]['major_discipline'].value_counts()[disc_order]
ax_side.barh(disc_no.index[::-1], disc_no[::-1], height=0.55, alpha=0.9,ec=background_color, color='gray')

disc_yes = data[data['target']==1]['major_discipline'].value_counts()[disc_order]
ax_side.barh(disc_yes.index[::-1], disc_yes[::-1], left= disc_no[::-1],height=0.55, alpha=0.9, ec=background_color,color='#0e4f66')

plt.setp(ax_side.get_yticklabels(), visible=False)
ax_side.set_facecolor(background_color)

# Spines
for s in ['top', 'left', 'right', 'bottom']:
    ax_centre.spines[s].set_visible(False)
    ax_top.spines[s].set_visible(False)
    ax_side.spines[s].set_visible(False)
ax_centre.set_axisbelow(True)    

fig.text(0.9, 0.9, 'Job seeking, company size, and major discipline', fontweight='bold', fontfamily='serif', fontsize=20, ha='right') 
fig.text(0.9, 0.87, 'Stacked Bar Chart & Categorical Bubble Pie Chart', fontweight='light', fontfamily='serif', fontsize=13, ha='right')

fig.text(0.633,0.84,"Job Seeking", fontweight="bold", fontfamily='serif', fontsize=15, color='#0e4f66')
fig.text(0.745,0.84,"|", fontweight="bold", fontfamily='serif', fontsize=15, color='black')
fig.text(0.755,0.84,"Not Job Seeking", fontweight="bold", fontfamily='serif', fontsize=15, color='gray')

ax_centre.tick_params(axis=u'both', which=u'both',length=0)
ax_top.tick_params(axis=u'both', which=u'both',length=0)
ax_side.tick_params(axis=u'both', which=u'both',length=0)

plt.tight_layout()
plt.show()

In [ ]:
# Orders
ed_order = ['Primary School','High School','Graduate','Masters','Phd','Unknown']
enroll_order = ['No Enrollment','Part time course','Full time course','Unknown']
disc_order = ['STEM','Humanities','Business Degree','Arts','Other','No Major','Unknown']
exp_yrs_order = ['0','<1','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','>20']
exp_yrs_order_2 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
size_order = ['0','<10', '10-49', '50-99', '100-500', '500-999', '1000-4999', '5000-9999', '10000+','Unemployed']
job_order = ['Never', '1', '2', '3', '4', '>4','Unknown']
exp_order =['No relevant experience','Has relevant experience']
gender_order = ['Male','Female','Other']
company_order = ['Pvt Ltd','Funded Startup','Public Sector','Early Stage Startup','NGO','Other','Unknown','Unemployed']

In [ ]:
job_change = aug_train.groupby(['education_level','last_new_job'])['experience'].sum().unstack().loc[ed_order,job_order]
jc_never = job_change['Never']


job_change_norm = pd.crosstab(aug_train['education_level'],aug_train['last_new_job'],normalize='columns').loc[ed_order,job_order,]
job_change_norm = round(job_change_norm*100,1).astype(int)




##
job_hunt_only = aug_train[aug_train['target']==1]
no_job_hunt_only = aug_train[aug_train['target']==0]


seekers_job_change_norm = pd.crosstab(job_hunt_only['education_level'],job_hunt_only['last_new_job'],normalize='columns').loc[ed_order,job_order,]
seekers_job_change_norm = round(seekers_job_change_norm*100,1).astype(int)
seekers_job_change_norm

non_seekers_job_change_norm = pd.crosstab(no_job_hunt_only['education_level'],no_job_hunt_only['last_new_job'],normalize='columns').loc[ed_order,job_order,]
non_seekers_job_change_norm = round(non_seekers_job_change_norm*100,1).astype(int)

## 새로운 EDA 여기서부터 시작!

In [ ]:
group_num = train_df[['enrollee_id', 'city_development_index', 'training_hours', 'target']]
group_obj = train_df[['gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size',
                       'company_type', 'last_new_job']]

## 실수형 변수 자료 분석

In [ ]:
corr_matrix = group_num.corr()

# 상관관계 히트맵 그리기
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

# Prediction

pip install scikit-learn==1.1
pip install imbalanced-learn==0.9

#### 여기서 범주형 자료로 변환 들어가야 함

##### 범주형 변수 전처리 
* one-hot encoding : gender, enrolled_university, major_discipline, company_type,relevent_experience
* ordinal encoding : education_level, company_size, last_new_job
* object -> int : Experience: 수치화

In [ ]:
# 모든 각 열에서의 unique한 value
for column in aug_train.columns:
    print(f"{column} unique values: \n{aug_train[column].unique()}\n")

##### scikit-learn 버전 출력
print(f"scikit-learn version: {sklearn.__version__}")
이 코드를 실행하여 scikit-learn의 버전을 확인하십시오. scikit-learn의 버전에 따라 OneHotEncoder의 매개변수 이름이 달라질 수 있습니다.

- 1.2.0 이상: sparse_output=False 사용
- 1.1.0 이하: sparse=False 사용

(버전 바꾸는 방법) 터미널에서 실행
-> pip uninstall scikit-learn -y
-> pip install scikit-learn==1.1.3


In [ ]:
import sklearn
print(f"scikit-learn version: {sklearn.__version__}")


In [ ]:
# One hot encoding 함수(dummy)
from sklearn.preprocessing import OneHotEncoder
## scikit-learn version: 1.4.2

In [ ]:

def one_hot_encode_column(df, column_name):
    # OneHotEncoder 객체 생성
    # sparse_output=False
    encoder = OneHotEncoder(drop='first', sparse=False, dtype=int)
    # 해당 칼럼에 대해 OneHotEncoding 수행
    encoded_data = encoder.fit_transform(df[[column_name]])
    
    # 새로운 칼럼 이름 생성
    new_column_names = encoder.get_feature_names_out([column_name])
    
    # 새로운 칼럼 데이터프레임 생성
    encoded_df = pd.DataFrame(encoded_data, columns=new_column_names, index=df.index)
    
    # 기존 열 제거 및 새로운 열 추가
    df = pd.concat([df, encoded_df], axis=1)
    df.drop(columns=[column_name], inplace=True)
    
    return df

In [ ]:
aug_train = one_hot_encode_column(aug_train, 'gender')
aug_train = one_hot_encode_column(aug_train, 'enrolled_university')
aug_train = one_hot_encode_column(aug_train, 'major_discipline')
aug_train = one_hot_encode_column(aug_train, 'company_type')
aug_train = one_hot_encode_column(aug_train, 'relevent_experience')

aug_train.head()

In [ ]:
aug_train.columns

In [ ]:
# Ordinal encoding 함수

def ordinal_encode_education_level(x):
    education_level_mapping = {'Primary School': 1, 'High School': 2, 'Graduate': 3, 'Masters': 4, 'Phd': 5, 'Unknown': 0}
    return education_level_mapping.get(x, x)

def ordinal_encode_company_size(x):
    company_size_mapping = {'<10': 1, '10-49': 2, '50-99': 3, '100-500': 4, '500-999': 5, '1000-4999': 6, '5000-9999': 7, '10000+': 8, 'Unknown': 0, 'Unemployed': 9}
    return company_size_mapping.get(x, x)

def ordinal_encode_last_new_job(x):
    last_new_job_mapping = {'Never': 1, '1': 2, '2': 3, '3': 4, '4': 5, '>4': 6, 'Unknown': 0}
    return last_new_job_mapping.get(x, x)

# education_level 칼럼에 ordinal_encode_education_level 함수를 적용하여 변환
aug_train['education_level'] = aug_train['education_level'].apply(ordinal_encode_education_level)

# company_size 칼럼에 ordinal_encode_company_size 함수를 적용하여 변환
aug_train['company_size'] = aug_train['company_size'].apply(ordinal_encode_company_size)

# last_new_job 칼럼에 ordinal_encode_last_new_job 함수를 적용하여 변환
aug_train['last_new_job'] = aug_train['last_new_job'].apply(ordinal_encode_last_new_job)


In [ ]:
# Unknown은 사실 ordinal에 들어가면 안되는데, 결측처리를 안했기 때문에 가지고 감
# 따라서 unknown인지 아닌지를 구분할 수 있는 더미칼럼을 생성

# 'education_level' 열을 기준으로 x가 0면 1, 아니면 0으로 'education_level_Unknown' 열 생성
aug_train['education_level_Unknown'] = aug_train['education_level'].apply(lambda x: 1 if x == 0 else 0)

# 'company_size' 열을 기준으로 x가 0이면 1, 아니면 0으로 'company_size_Unknown' 열 생성
aug_train['company_size_Unknown'] = aug_train['company_size'].apply(lambda x: 1 if x == 0 else 0)

# 'company_size' 열을 기준으로 x가 9이면 1, 아니면 0으로 'company_size_Unknown' 열 생성
aug_train['company_size_Unemployed'] = aug_train['company_size'].apply(lambda x: 1 if x == 9 else 0)

# 'last_new_job' 열을 기준으로 x가 0이면 1, 아니면 0으로 'last_new_job_Unknown' 열 생성
aug_train['last_new_job_Unknown'] = aug_train['last_new_job'].apply(lambda x: 1 if x == 0 else 0)

In [ ]:
## 더미를 만드는 대신 원래 있던 열에서는 0값으로 변경(추가된 코드)
# 'education_level'가 0인 행의 'education_level'를 0로 변경
aug_train.loc[aug_train['education_level'] == 0, 'education_level'] = 0

# 'company_size'가 0인 행의 'education_level'를 0로 변경
aug_train.loc[aug_train['company_size'] == 0, 'company_size'] = 0

# 'company_size'가 9인 행의 'education_level'를 0로 변경
aug_train.loc[aug_train['company_size'] == 9, 'company_size'] = 0

# 'last_new_job'가 0인 행의 'last_new_job'를 0로 변경
aug_train.loc[aug_train['last_new_job'] == 0, 'last_new_job'] = 0

In [ ]:
aug_train['company_size'] = aug_train['company_size'].astype(int)

In [ ]:
print(aug_train['education_level'].unique())
print(aug_train['company_size'].unique())
print(aug_train['last_new_job'].unique())

In [ ]:
aug_train.info()

In [ ]:
aug_train[['company_size_Unemployed','company_type_Unemployed']].value_counts()

In [ ]:
aug_train.drop(columns='company_size_Unemployed',axis=1, inplace=True)

In [ ]:
aug_train.columns

##### 모델 전에 마지막 작업

In [ ]:
aug_train = aug_train.drop(columns=['enrollee_id','count'], axis=1)

# Final look at our df
aug_train.head(3)

In [ ]:
# model 돌리기 전에 최종 데이터 셋 정보
aug_train.info()

# Modeling

In [ ]:
from sklearn.model_selection import train_test_split

X = aug_train.dropna().drop(columns=['target']).values
y = aug_train.dropna()['target'].values

# train, val, test 셋 분리하는 코드
# 데이터셋 분할: Train/Validation + Test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=80)
# 데이터셋 분할: Train + Validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=80)

In [ ]:
# 열 이름 설정
feature_columns = aug_train.drop(columns=['target']).columns
target_column = ['target']

# numpy 배열을 DataFrame으로 변환
X_train_df = pd.DataFrame(X_train_val, columns=feature_columns)
X_val_df = pd.DataFrame(X_val, columns=feature_columns)
X_test_df = pd.DataFrame(X_test, columns=feature_columns)
y_train_df = pd.DataFrame(y_train_val, columns=target_column)
y_val_df = pd.DataFrame(y_val, columns=target_column)
y_test_df = pd.DataFrame(y_test, columns=target_column)

# Ensure 'train_df' and 'test_df' are created correctly by concatenating X and y
# train_df : for visuals
train_df = pd.concat([X_train_df, y_train_df], axis=1)
test_df = pd.concat([X_val_df, y_val_df], axis=1)
new_data_df = pd.concat([X_test_df, y_test_df], axis=1)

# 출력 확인 
print(train_df.info())
print(test_df.info())

In [ ]:
X_train, X_test, y_train, y_test = X_train, X_val, y_train, y_val

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score, confusion_matrix

from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

## LinearSVC

In [ ]:
linearSVC = LinearSVC(dual=False, random_state=123, max_iter=10000)
linearSVC.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
linearSVC_prediction = linearSVC.predict(X_test_scaled)
cm_linearSVC = confusion_matrix(y_test, linearSVC_prediction)
df_linearSVC = pd.DataFrame(data=[f1_score(y_test, linearSVC_prediction), recall_score(y_test, linearSVC_prediction),
                   precision_score(y_test, linearSVC_prediction), accuracy_score(y_test, linearSVC_prediction), 
                   roc_auc_score(y_test, linearSVC_prediction)],
             columns=['LinearSVC Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_linearSVC

In [ ]:
params_grid = {
    "C": [10 ** k for k in range(-3, 4)], # 1.0
    "penalty": ['l2', 'l1'], # l2
    # "tol": [1e-4, 1e-3, 1e-2] # 1e-4
}

# Initialize the GridSearchCV with LinearSVC
linearSVC_t = GridSearchCV(linearSVC, params_grid, cv=5, scoring='f1')
linearSVC_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", linearSVC_t.best_params_)

# Predictions and evaluation
linearSVC_t_prediction = linearSVC_t.predict(X_test_scaled)
cm_linearSVC_t = confusion_matrix(y_test, linearSVC_t_prediction)
df_linearSVC_t = pd.DataFrame(data=[f1_score(y_test, linearSVC_t_prediction), recall_score(y_test, linearSVC_t_prediction),
                   precision_score(y_test, linearSVC_t_prediction), accuracy_score(y_test, linearSVC_t_prediction), 
                   roc_auc_score(y_test, linearSVC_t_prediction)],
             columns=['Tuned LinearSVC Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_linearSVC_t

## SVM

### sigmoid

In [ ]:
SVC_sigmoid = SVC(kernel='sigmoid')
SVC_sigmoid.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
SVC_sigmoid_prediction = SVC_sigmoid.predict(X_test_scaled)
cm_SVC_sigmoid = confusion_matrix(y_test, SVC_sigmoid_prediction)
df_SVC_sigmoid = pd.DataFrame(data=[f1_score(y_test, SVC_sigmoid_prediction), recall_score(y_test, SVC_sigmoid_prediction),
                   precision_score(y_test, SVC_sigmoid_prediction), accuracy_score(y_test, SVC_sigmoid_prediction), 
                   roc_auc_score(y_test, SVC_sigmoid_prediction)],
             columns=['SVC Sigmoid Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_sigmoid

In [ ]:
params_grid = {
    'C': [10 ** k for k in range(-3, 3)], # 1.0
    'gamma': ['scale', 'auto', 0.01, 0.1, 1] # scale
}

# Initialize the GridSearchCV
SVC_sigmoid_t = GridSearchCV(SVC_sigmoid, params_grid, cv=5, scoring='f1')
SVC_sigmoid_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", SVC_sigmoid_t.best_params_)

# Predictions and evaluation
SVC_sigmoid_t_prediction = SVC_sigmoid_t.predict(X_test_scaled)
cm_SVC_sigmoid_t = confusion_matrix(y_test, SVC_sigmoid_t_prediction)
df_SVC_sigmoid_t = pd.DataFrame(data=[f1_score(y_test, SVC_sigmoid_t_prediction), recall_score(y_test, SVC_sigmoid_t_prediction),
                   precision_score(y_test, SVC_sigmoid_t_prediction), accuracy_score(y_test, SVC_sigmoid_t_prediction), 
                   roc_auc_score(y_test, SVC_sigmoid_t_prediction)],
             columns=['Tuned SVC Sigmoid Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_sigmoid_t

### Poly

In [ ]:
SVC_poly = SVC(kernel='poly')
SVC_poly.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
SVC_poly_prediction = SVC_poly.predict(X_test_scaled)
cm_SVC_poly = confusion_matrix(y_test, SVC_poly_prediction)
df_SVC_poly = pd.DataFrame(data=[f1_score(y_test, SVC_poly_prediction), recall_score(y_test, SVC_poly_prediction),
                   precision_score(y_test, SVC_poly_prediction), accuracy_score(y_test, SVC_poly_prediction), 
                   roc_auc_score(y_test, SVC_poly_prediction)],
             columns=['SVC Poly Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_poly

In [ ]:
params_grid = {
    'C': [10 ** k for k in range(-3, 3)],    
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    # 'degree': [2, 3, 4, 5] # 3
}

# Initialize the GridSearchCV
SVC_poly_t = GridSearchCV(SVC_poly, params_grid, cv=5, scoring='f1')
SVC_poly_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", SVC_poly_t.best_params_)

# Predictions and evaluation
SVC_poly_t_prediction = SVC_poly_t.predict(X_test_scaled)
cm_SVC_poly_t = confusion_matrix(y_test, SVC_poly_t_prediction)
df_SVC_poly_t = pd.DataFrame(data=[f1_score(y_test, SVC_poly_t_prediction), recall_score(y_test, SVC_poly_t_prediction),
                   precision_score(y_test, SVC_poly_t_prediction), accuracy_score(y_test, SVC_poly_t_prediction), 
                   roc_auc_score(y_test, SVC_poly_t_prediction)],
             columns=['Tuned SVC Poly Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_poly_t

### RBF

In [ ]:
SVC_rbf = SVC(kernel='rbf')
SVC_rbf.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
SVC_RBF_prediction = SVC_rbf.predict(X_test_scaled)
cm_SVC_RBF = confusion_matrix(y_test, SVC_RBF_prediction)
df_SVC_RBF = pd.DataFrame(data=[f1_score(y_test, SVC_RBF_prediction), recall_score(y_test, SVC_RBF_prediction),
                   precision_score(y_test, SVC_RBF_prediction), accuracy_score(y_test, SVC_RBF_prediction), 
                   roc_auc_score(y_test, SVC_RBF_prediction)],
             columns=['SVC RBF Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_RBF

In [ ]:
params_grid = {
    'C': [10 ** k for k in range(-3, 3)],    
    'gamma': ['scale', 'auto', 0.01, 0.1, 1]
}

# Initialize the GridSearchCV
SVC_rbf_t = GridSearchCV(SVC_rbf, params_grid, cv=5, scoring='f1')
SVC_rbf_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", SVC_rbf_t.best_params_)

# Predictions and evaluation
SVC_RBF_t_prediction = SVC_rbf_t.predict(X_test_scaled)
cm_SVC_RBF_t = confusion_matrix(y_test, SVC_RBF_t_prediction)
df_SVC_RBF_t = pd.DataFrame(data=[f1_score(y_test, SVC_RBF_t_prediction), recall_score(y_test, SVC_RBF_t_prediction),
                   precision_score(y_test, SVC_RBF_t_prediction), accuracy_score(y_test, SVC_RBF_t_prediction), 
                   roc_auc_score(y_test, SVC_RBF_t_prediction)],
             columns=['Tuned SVC RBF Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_RBF_t

### Linear

In [ ]:
SVC_linear = SVC(kernel='linear')
SVC_linear.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
SVC_linear_prediction = SVC_linear.predict(X_test_scaled)
cm_SVC_linear = confusion_matrix(y_test, SVC_linear_prediction)
df_SVC_linear = pd.DataFrame(data=[f1_score(y_test, SVC_linear_prediction), recall_score(y_test, SVC_linear_prediction),
                   precision_score(y_test, SVC_linear_prediction), accuracy_score(y_test, SVC_linear_prediction), 
                   roc_auc_score(y_test, SVC_linear_prediction)],
             columns=['SVC Linear Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_linear

In [ ]:
params_grid = {
    'C': [10 ** k for k in range(-3, 3)],
}

# Initialize the GridSearchCV
SVC_linear_t = GridSearchCV(SVC_linear, params_grid, cv=5, scoring='f1')
SVC_linear_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", SVC_linear_t.best_params_)

# Predictions and evaluation
SVC_linear_t_prediction = clf.predict(X_test_scaled)
cm_SVC_linear_t = confusion_matrix(y_test, SVC_linear_t_prediction)
df_SVC_linear_t = pd.DataFrame(data=[f1_score(y_test, SVC_linear_t_prediction), recall_score(y_test, SVC_linear_t_prediction),
                   precision_score(y_test, SVC_linear_t_prediction), accuracy_score(y_test, SVC_linear_t_prediction), 
                   roc_auc_score(y_test, SVC_linear_t_prediction)],
             columns=['Tuned SVC RBF Kernel Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_SVC_linear_t

## Decision Tree

In [ ]:
dtree = DecisionTreeClassifier(random_state=123)
dtree.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
dtree_prediction = dtree.predict(X_test_scaled)
cm_dtree = confusion_matrix(y_test, dtree_prediction)
df_dtree = pd.DataFrame(data=[f1_score(y_test, dtree_prediction), recall_score(y_test, dtree_prediction),
                   precision_score(y_test, dtree_prediction), accuracy_score(y_test, dtree_prediction), 
                   roc_auc_score(y_test, dtree_prediction)],
             columns=['Decision Tree Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_dtree

In [ ]:
params_grid = {
    "max_depth": [None, 10, 20, 30, 40], # None
    "min_samples_split": [2, 10, 20, 50], # 2
    # "min_samples_leaf": [1, 2, 5, 10], # 1
    # "criterion": ['gini', 'entropy', 'log_loss'] # gini
}

# Initialize the GridSearchCV
dtree_t = GridSearchCV(dtree, params_grid, cv=5, scoring='f1')
dtree_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", dtree_t.best_params_)

# Predictions and evaluation
dtree_t_prediction = dtree_t.predict(X_test_scaled)
cm_dtree_t = confusion_matrix(y_test, dtree_t_prediction)
df_dtree_t = pd.DataFrame(data=[f1_score(y_test, dtree_t_prediction), recall_score(y_test, dtree_t_prediction),
                   precision_score(y_test, dtree_t_prediction), accuracy_score(y_test, dtree_t_prediction), 
                   roc_auc_score(y_test, dtree_t_prediction)],
             columns=['Tuned Decision Tree Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_dtree_t

## Random Forest

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
rfc_prediction = rfc.predict(X_test_scaled)
cm_rfc = confusion_matrix(y_test, rfc_prediction)
df_rfc = pd.DataFrame(data=[f1_score(y_test, rfc_prediction), recall_score(y_test, rfc_prediction),
                   precision_score(y_test, rfc_prediction), accuracy_score(y_test, rfc_prediction), 
                   roc_auc_score(y_test, rfc_prediction)],
             columns=['Random Forest Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_dtree

In [ ]:
params_grid = {
    'n_estimators': [100, 200, 300, 400, 500, 600], # 100
    'max_depth': [10, 20, 30, 40, 50, None], # None
    'min_samples_split': [2, 5, 10, 20], # 2
    'max_features': ['sqrt', 'log2', None], # auto
}

# Initialize the GridSearchCV
rfc_t = GridSearchCV(rfc, params_grid, cv=5, scoring='f1')
rfc_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", rfc_t.best_params_)

# Predictions and evaluation
rfc_t_prediction = rfc_t.predict(X_test_scaled)
cm_rfc_t = confusion_matrix(y_test, rfc_t_prediction)
df_rfc_t = pd.DataFrame(data=[f1_score(y_test, rfc_t_prediction), recall_score(y_test, rfc_t_prediction),
                   precision_score(y_test, rfc_t_prediction), accuracy_score(y_test, rfc_t_prediction), 
                   roc_auc_score(y_test, rfc_t_prediction)],
             columns=['Tuned Random Forest Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_rfc_t

random_state=0, n_estimators= 800, criterion = 'gini',max_features = 'sqrt',max_depth = 8

## Logistic Regression

In [ ]:
logistic = LogisticRegression()
logistic.fit(X_train_resampled, y_train_resampled)

# 예측 및 평가
log_prediction = logistic.predict(X_test_scaled)
cm_log = confusion_matrix(y_test, log_prediction)
df_log = pd.DataFrame(data=[f1_score(y_test, log_prediction), recall_score(y_test, log_prediction),
                   precision_score(y_test, log_prediction), accuracy_score(y_test, log_prediction), 
                   roc_auc_score(y_test, log_prediction)],
             columns=['Logistic Regression Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_log

In [ ]:
params_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # 'C' 파라미터 값
    'penalty': ['l1', 'l2', 'elasticnet'],  # 'penalty' 옵션
    'solver': ['saga'],  # 모든 페널티 유형을 지원
    'l1_ratio': [0, 0.5, 1]  # elasticnet 사용 시 필요한 l1_ratio 값
}

# Initialize the GridSearchCV
logistic_t = GridSearchCV(logistic, params_grid, cv=5, scoring='f1')
logistic_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", logistic_t.best_params_)

# Predictions and evaluation
rfc_t_prediction = logistic_t.predict(X_test_scaled)
cm_rfc_t = confusion_matrix(y_test, rfc_t_prediction)
df_rfc_t = pd.DataFrame(data=[f1_score(y_test, rfc_t_prediction), recall_score(y_test, rfc_t_prediction),
                   precision_score(y_test, rfc_t_prediction), accuracy_score(y_test, rfc_t_prediction), 
                   roc_auc_score(y_test, rfc_t_prediction)],
             columns=['Tuned Random Forest Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_rfc_t

## KNN

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

knn_prediction = knn.predict(X_test_scaled)
cm_knn = confusion_matrix(y_test, knn_prediction)
df_knn = pd.DataFrame(data=[f1_score(y_test, knn_prediction), recall_score(y_test, knn_prediction),
                   precision_score(y_test, knn_prediction), accuracy_score(y_test, knn_prediction), 
                   roc_auc_score(y_test, knn_prediction)],
             columns=['KNN Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_knn

In [ ]:
params_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 17], # 5
    'weights': ['uniform', 'distance'], # uniform
    # 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], # auto
    # 'leaf_size': [10, 20, 30, 40, 50] # 30
}

# Initialize the GridSearchCV
knn_t = GridSearchCV(knn, params_grid, cv=5, scoring='f1')
knn_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", knn_t.best_params_)

# Predictions and evaluation
knn_t_prediction = knn_t.predict(X_test_scaled)
cm_knn_t = confusion_matrix(y_test, knn_t_prediction)
df_knn_t = pd.DataFrame(data=[f1_score(y_test, knn_t_prediction), recall_score(y_test, knn_t_prediction),
                   precision_score(y_test, knn_t_prediction), accuracy_score(y_test, knn_t_prediction), 
                   roc_auc_score(y_test, knn_t_prediction)],
             columns=['Tuned KNN Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_knn_t

## XGBoost

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

xgb_prediction = xgb.predict(X_test_scaled)
cm_xgb = confusion_matrix(y_test, xgb_prediction)
df_xgb = pd.DataFrame(data=[f1_score(y_test, xgb_prediction), recall_score(y_test, xgb_prediction),
                   precision_score(y_test, xgb_prediction), accuracy_score(y_test, xgb_prediction), 
                   roc_auc_score(y_test, xgb_prediction)],
             columns=['XGBoost Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_xgb

In [ ]:
params_grid = {
    'n_estimators': [100, 200, 300, 400, 500], # 100
    'max_depth': [3, 5, 6, 7, 9, 11], # 6
    # 'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3], # 0.3
    # 'subsample': [0.6, 0.7, 0.8, 0.9, 1.0] # 1.0
}

# Initialize the GridSearchCV
xgb_t = GridSearchCV(xgb, params_grid, cv=5, scoring='f1')
xgb_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", xgb_t.best_params_)

# Predictions and evaluation
xgb_t_prediction = xgb_t.predict(X_test_scaled)
cm_xgb_t = confusion_matrix(y_test, xgb_t_prediction)
df_xgb_t = pd.DataFrame(data=[f1_score(y_test, xgb_t_prediction), recall_score(y_test, xgb_t_prediction),
                   precision_score(y_test, xgb_t_prediction), accuracy_score(y_test, xgb_t_prediction), 
                   roc_auc_score(y_test, xgb_t_prediction)],
             columns=['Tuned XGBoost Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_xgb_t

## AdaBoost

In [ ]:
adab = AdaBoostClassifier()
adab.fit(X_train,y_train)

adab_prediction = adab.predict(X_test_scaled)
cm_adab = confusion_matrix(y_test, adab_prediction)
df_adab = pd.DataFrame(data=[f1_score(y_test, adab_prediction), recall_score(y_test, adab_prediction),
                   precision_score(y_test, adab_prediction), accuracy_score(y_test, adab_prediction), 
                   roc_auc_score(y_test, adab_prediction)],
             columns=['AdaBoost Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_adab

In [ ]:
params_grid = {
    'n_estimators': [50, 100, 200, 300, 400, 500], # 50
    'learning_rate': [0.01, 0.1, 0.5, 1.0], # 1.0
    # 'algorithm': ['SAMME', 'SAMME.R'], # SAMME.R
    # 'base_estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2)] # DecisionTreeClassifier(max_depth=1)
}

# Initialize the GridSearchCV
adab_t = GridSearchCV(adab, params_grid, cv=5, scoring='f1')
adab_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", adab_t.best_params_)

# Predictions and evaluation
xgb_t_prediction = adab_t.predict(X_test_scaled)
cm_xgb_t = confusion_matrix(y_test, xgb_t_prediction)
df_xgb_t = pd.DataFrame(data=[f1_score(y_test, xgb_t_prediction), recall_score(y_test, xgb_t_prediction),
                   precision_score(y_test, xgb_t_prediction), accuracy_score(y_test, xgb_t_prediction), 
                   roc_auc_score(y_test, xgb_t_prediction)],
             columns=['Tuned AdaBoost Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_xgb_t

## Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train,y_train)

nb_prediction = nb.predict(X_test_scaled)
cm_nb = confusion_matrix(y_test, nb_prediction)
df_nb = pd.DataFrame(data=[f1_score(y_test, nb_prediction), recall_score(y_test, nb_prediction),
                   precision_score(y_test, nb_prediction), accuracy_score(y_test, nb_prediction), 
                   roc_auc_score(y_test, nb_prediction)],
             columns=['Naive Bayes Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_nb

In [ ]:
params_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6], # 1e-9
}

# Initialize the GridSearchCV
nb_t = GridSearchCV(nb, params_grid, cv=5, scoring='f1')
nb_t.fit(X_train_resampled, y_train_resampled)

# Output the best parameters
print("Best Parameters: ", nb_t.best_params_)

# Predictions and evaluation
nb_t_prediction = nb_t.predict(X_test_scaled)
cm_nb_t = confusion_matrix(y_test, nb_t_prediction)
df_nb_t = pd.DataFrame(data=[f1_score(y_test, nb_t_prediction), recall_score(y_test, nb_t_prediction),
                   precision_score(y_test, nb_t_prediction), accuracy_score(y_test, nb_t_prediction), 
                   roc_auc_score(y_test, nb_t_prediction)],
             columns=['Tuned Naive Bayes Score'],
             index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"])
df_nb_t

# Test Data

In [ ]:
# Assume 'target' is the target column and the rest are features
X_new_test = new_data_df.drop(columns=['target'])
y_new_test = new_data_df['target']

# Convert to numpy arrays for consistency
X_new_test_array = X_new_test.values

# 전처리 (Scaling) 적용
scaler = StandardScaler()
X_new_test_scaled = scaler.fit_transform(X_new_test_array)

In [ ]:
def test_data(model, model_t):
    prediction = model.predict(X_new_test_scaled)
    prediction_t = model_t.predict(X_new_test_scaled)

    cm_new = confusion_matrix(y_new_test, prediction)
    df_new = pd.DataFrame(
        data=[
            f1_score(y_new_test, prediction),
            recall_score(y_new_test, prediction),
            precision_score(y_new_test, prediction),
            accuracy_score(y_new_test, prediction),
            roc_auc_score(y_new_test, prediction)
        ],
        columns=[f'{model} Score'],
        index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"]
    )

    cm_new_t = confusion_matrix(y_new_test, prediction_t)
    df_new_t = pd.DataFrame(
        data=[
            f1_score(y_new_test, prediction_t),
            recall_score(y_new_test, prediction_t),
            precision_score(y_new_test, prediction_t),
            accuracy_score(y_new_test, prediction_t),
            roc_auc_score(y_new_test, prediction_t)
        ],
        columns=[f'Tuned {model_t} Score'],
        index=["F1 Score", "Recall", "Precision", "Accuracy", "ROC AUC Score"]
    )

    # 결과 출력
    print("Model result")
    print(cm_new)
    print(df_new)

    # 결과 출력
    print("Tuned Model result")
    print(cm_new_t)
    print(df_new_t)


In [ ]:
test_data(logistic, logistic_t)

linearSVC, SVC_sigmoid, SVC_poly, SVC_rbf, SVC_linear, dtree, rfc, logistic, knn, xgb, adab, nb

# So far

In [ ]:
fig = plt.figure(figsize=(15,18), dpi=150)
gs = fig.add_gridspec(4, 2)
gs.update(wspace=0.1, hspace=0.5)
ax0 = fig.add_subplot(gs[0, :])

colors = ["lightgray","lightgray"]
colormap = matplotlib.colors.LinearSegmentedColormap.from_list("", colors)
# Change background color
background_color = "#fbfbfb"
fig.patch.set_facecolor(background_color) # figure background color
ax0.set_facecolor(background_color)

# Overall
df_models = round(pd.concat([df_linearSVC, df_SVC_sigmoid, df_SVC_linear, df_SVC_RBF, df_SVC_poly,
                             df_dtree, df_rfc, df_knn, df_xgb, df_adab, df_nb], axis=1),3)
sns.heatmap(df_models.T, cmap=colormap,annot=True,fmt=".1%", linewidths=2.5,cbar=False,ax=ax0)

ax0.tick_params(axis=u'both', which=u'both',length=0)
ax0.text(0,-2,'Our results so far',fontfamily='serif',fontsize=20,fontweight='bold')

from matplotlib.patches import Rectangle

rect = ax0.add_patch(Rectangle((0, 0), 5, 1, fill=True,color='#0e4f66', edgecolor='white', lw=0,alpha=0.5))
rect = ax0.add_patch(Rectangle((0, 2), 5, 1, fill=True,color='#0e4f66', edgecolor='white', lw=0,alpha=0.5))



plt.show()